In [24]:
import numpy as np
import napari

import matplotlib.pyplot as plt

from skimage.io import imread, imsave
from skimage import exposure, restoration

from tqdm.notebook import tqdm

In [25]:
%matplotlib inline

## Start napari

In [26]:
viewer = napari.Viewer()

C:\Users\KarenDuffGroup\AppData\Roaming\Python\Python38\site-packages\napari_tools_menu\__init__.py:165: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


## Load image and add channels to Napari

In [27]:
# Path on analysis computer
im = imread('E:mathieu/220222_full_hipp_IF/full.tif')

In [ ]:
# Path on laptop
im = imread('../mc1_cp27_p62_hipp_stitched.tif')

In [28]:
im = im[1:, :, :, :]
im.shape

(26, 28, 14452, 10648)

In [55]:
cropped = im[:, :, ymin:ymax, xmin:xmax]

In [56]:
viewer.add_image(cropped[:, 0, :, :], name='dapi', blending='additive', 
                 colormap='bop blue', scale=[1,1,1])

<Image layer 'dapi' at 0x212060244c0>

In [57]:
viewer.add_image(cropped[:, 23, :, :], name='mc1', blending='additive', 
                 colormap='bop orange', scale=[1,1,1])

viewer.add_image(cropped[:, 18, :, :], name='anchor', blending='additive', 
                 colormap='bop purple', scale=[1,1,1])

viewer.add_image(cropped[:, 27, :, :], name='p62', blending='additive', 
                 scale=[1,1,1])

<Image layer 'p62' at 0x212169455b0>

In [51]:
viewer.add_image(im[:, 0, :, :], name='dapi', blending='additive', colormap='bop blue', scale=[1,1,1])

<Image layer 'dapi' at 0x2122eaea190>

In [6]:
viewer.add_image(im[:, 9, :, :], name='cp27', blending='additive', colormap='bop orange', scale=[1,1,1])

<Image layer 'cp27' at 0x2122cba0640>

In [7]:
viewer.add_image(im[:, 27, :, :], name='p62', blending='additive', scale=[1,1,1])

<Image layer 'p62' at 0x2123a40c3a0>

In [8]:
viewer.add_image(im[:, 18, :, :], name='anchor', blending='additive', scale=[1,1,1])

<Image layer 'anchor' at 0x212050e11f0>

In [9]:
viewer.add_image(im[:, 23, :, :], name='mc1', blending='additive', colormap='bop purple', scale=[1,1,1])

<Image layer 'mc1' at 0x2121a8ea310>

In [10]:
viewer.add_image(im[:, 6, :, :], name='lf', blending='additive', colormap='cyan', scale=[1,1,1])

<Image layer 'lf' at 0x21227259970>

In [12]:
viewer.screenshot('E:mathieu/220222_full_hipp_IF/dapi_screenshot.png');

## Load nuclear segmentation

In [ ]:
# Reload segmentation
dat = np.load('../full_hipp_seg.npy', allow_pickle=True).item()

In [ ]:
viewer.add_labels(dat['masks'])

## Improve picture quality

In [ ]:
mc1_bckgnd = restoration.rolling_ball(im[:, 9, :, :])

# Illustration

In [13]:
# Import napari colormaps

from matplotlib.colors import ListedColormap

bopblue = ListedColormap(napari.utils.colormaps.bop_colors.bop_blue)
boporange = ListedColormap(napari.utils.colormaps.bop_colors.bop_orange)
boppurple = ListedColormap(napari.utils.colormaps.bop_colors.bop_purple)
#cyan = napari.utils.colormaps.SIMPLE_COLORMAPS['cyan']

In [14]:
def plot_style(figsize=(6, 12), labelsize=12, titlesize=24, ticklabelsize=12, **kwargs):
    basic_style = {
       'figure.figsize': figsize,
       'axes.labelsize': labelsize,
       'axes.titlesize': titlesize,
       'xtick.bottom' : False,
        'xtick.labelbottom': False,
       'ytick.left' : False,
        'ytick.labelleft' : False,
       #'xtick.labelsize': ticklabelsize,
       #'ytick.labelsize': ticklabelsize,
       'axes.spines.top': False,
       'axes.spines.bottom': False,
       'axes.spines.right': False,
       'axes.spines.left': False,
       'axes.grid': False,
       
       #Distance between plots
       'figure.subplot.bottom': 0.1,
       'figure.subplot.hspace': 0.05,
       'figure.subplot.left': 0.1,
       'figure.subplot.right': 0.9,
       'figure.subplot.top': 0.9,
       'figure.subplot.wspace': 0.05,    
       
   }
    basic_style.update(kwargs)
    return plt.rc_context(rc=basic_style)

# If looking for more parameters check in plt.rcParams.keys()

In [52]:
#Create layer to select ROI
viewer.add_shapes(name='ROI', face_color='r')

C:\Users\KarenDuffGroup\AppData\Roaming\Python\Python38\site-packages\numpy\core\numeric.py:2446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


<Shapes layer 'ROI' at 0x2121b365d90>

In [53]:
# Crop image using rectangle ROI
roi_coords = viewer.layers['ROI'].data[0]

In [54]:
ymax, xmax = int(roi_coords.max(axis=0)[0]), int(roi_coords.max(axis=0)[1])
ymin, xmin = int(roi_coords.min(axis=0)[0]), int(roi_coords.min(axis=0)[1])

In [35]:
cropped = im[14, :, ymin:ymax, xmin:xmax]

In [23]:
with plot_style(figsize=(10,9)):

    plt.subplot(321)

    dapi = np.swapaxes(cropped[0, :, :], 0, 1)
    dapi_pct = np.percentile(dapi, (10, 99.5))
    plt.imshow(exposure.rescale_intensity(dapi, in_range=tuple(dapi_pct)), cmap=bopblue)
    plt.text(x=50, y=120, s="DAPI (405)", fontdict={'font': 'Verdana', 'size': 20, 'color': 'w'})

    plt.subplot(322)
    cp27 = np.swapaxes(cropped[9, :, :], 0, 1)
    cp27_pct = np.percentile(cp27, (10, 99.5))
    plt.imshow(exposure.rescale_intensity(cp27, in_range=tuple(cp27_pct)), cmap=boporange)
    plt.text(x=50, y=120, s="CP27 (488)", fontdict={'font': 'Verdana', 'size': 20, 'color': 'w'})
    
    plt.subplot(323)
    anchor = np.swapaxes(cropped[18, :, :], 0, 1)
    anchor_pct = np.percentile(anchor, (10, 99.5))
    plt.imshow(exposure.rescale_intensity(anchor, in_range=tuple(anchor_pct)), cmap=boppurple)
    plt.text(x=50, y=120, s="Anchor (555)", fontdict={'font': 'Verdana', 'size': 20, 'color': 'w'})

    plt.subplot(324)
    mc1 = np.swapaxes(cropped[23, :, :], 0, 1)
    mc1_pct = np.percentile(mc1, (10, 99.5))
    plt.imshow(exposure.rescale_intensity(mc1, in_range=tuple(mc1_pct)), cmap='Greys_r')
    plt.text(x=50, y=120, s="MC1 (640)", fontdict={'font': 'Verdana', 'size': 20, 'color': 'w'})

    plt.subplot(325)
    p62 = np.swapaxes(cropped[27, :, :], 0, 1)
    p62_pct = np.percentile(p62, (20, 99.5))
    plt.imshow(exposure.rescale_intensity(p62, in_range=tuple(p62_pct)), cmap='Greys_r')
    plt.text(x=50, y=120, s="p62(750)", fontdict={'font': 'Verdana', 'size': 20, 'color': 'w'})
    
    plt.subplot(326)
    lf = np.swapaxes(cropped[6, :, :], 0, 1)
    lf_pct = np.percentile(lf, (20, 99.5))
    plt.imshow(exposure.rescale_intensity(lf, in_range=tuple(lf_pct)), cmap='Greys_r')
    plt.text(x=50, y=120, s="LF", fontdict={'font': 'Verdana', 'size': 20, 'color': 'w'})

plt.tight_layout()
plt.savefig('E:mathieu/220222_full_hipp_IF/6color.png', dpi=300)

plt.show()

## Antibody diffusion along Z

In [ ]:
yz = im[:, :, ymin:ymax, xmin:xmax].sum(axis=3)

In [ ]:
# Swap axis between color channel (1), and zplanes (0) to avoid change the plot code below 
yz = np.moveaxis(yz, [0, 1], [1, 0])

In [ ]:
with plot_style(figsize=(15,5)):
    
    # Position fo the dye legend
    leg_x, leg_y = 5, 12
    
    plt.subplot(511)

    dapi = yz[0, :, :]
    dapi_pct = np.percentile(dapi, (10, 99.5))
    plt.imshow(exposure.rescale_intensity(dapi, in_range=tuple(dapi_pct)), extent=[0, 592, 0, 20], aspect='auto')
    plt.text(x=leg_x, y=leg_y, s="DAPI (405)", fontdict={'font': 'Verdana', 'size': 20, 'color': 'w'})

    plt.subplot(512)
    mc1 = yz[9, :, :]
    mc1_pct = np.percentile(dapi, (10, 99.5))
    plt.imshow(exposure.rescale_intensity(mc1, in_range=tuple(mc1_pct)), 
               extent=[0, 592, 0, 20], aspect='auto')
    plt.text(x=leg_x, y=leg_y, s="MC1 (488)", fontdict={'font': 'Verdana', 'size': 20, 'color': 'w'})
    
    plt.subplot(513)
    cp27 = yz[19, :, :]
    cp27_pct = np.percentile(cp27, (10, 99.5))
    plt.imshow(exposure.rescale_intensity(cp27, in_range=tuple(cp27_pct)), extent=[0, 592, 0, 20], aspect='auto')
    plt.text(x=leg_x, y=leg_y, s="CP27 (640)", fontdict={'font': 'Verdana', 'size': 20, 'color': 'w'})

    plt.subplot(514)
    p62 = yz[14, :, :]
    p62_pct = np.percentile(p62, (10, 99.5))
    plt.imshow(exposure.rescale_intensity(p62, in_range=tuple(p62_pct)), extent=[0, 592, 0, 20], aspect='auto')
    plt.text(x=leg_x, y=leg_y, s="p62 (555)", fontdict={'font': 'Verdana', 'size': 20, 'color': 'w'})

    plt.subplot(515)
    lf = yz[6, :, :]
    lf_pct = np.percentile(lf, (20, 99.5))
    plt.imshow(exposure.rescale_intensity(lf, in_range=tuple(lf_pct)), extent=[0, 592, 0, 20], aspect='auto')
    plt.text(x=leg_x, y=leg_y, s="LF", fontdict={'font': 'Verdana', 'size': 20, 'color': 'w'})


plt.savefig('xz_plots.pdf', dpi=300)

plt.show()